In [1]:
!pip install numpy==1.24.3
!pip install scikit-surprise

  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505218 sha256=3cb807f0eac7369998a0ca7db9c264e590f0b3b621168d96ae52b777f7a707f9
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [2]:
# from surprise import accuracy, Dataset, SVD
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from surprise import NormalPredictor, Reader
from surprise.model_selection import train_test_split
import pickle
from google.colab import files
import numpy as np
from sklearn.preprocessing import normalize

In [4]:
df_1 = pd.read_csv("movies.csv")
df_2 = pd.read_csv("ratings.csv")

df_1.columns = ["movieId", "title", "genres"]
df_2.columns = ["userId", "movieId", "rating", "timestamp"]

df_2.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
df_1.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
df = df_2.join(df_1, on=["movieId"], lsuffix='', rsuffix='_456')
df.pop('movieId_456')
df.pop('timestamp')

df.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Jumanji (1995),Adventure|Children|Fantasy
1,1,3,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance
2,1,6,4.0,Sabrina (1995),Comedy|Romance
3,1,47,5.0,Mighty Aphrodite (1995),Comedy|Drama|Romance
4,1,50,5.0,Georgia (1995),Drama


In [7]:
df_ohe = df['genres'].str.get_dummies(sep='|')

df = pd.concat([df, df_ohe], axis = 1)
df

,userId,movieId,rating,title,genres,(no genres listed),Action,Adventure,Animation,Children,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,Sabrina (1995),Comedy|Romance,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1,47,5.0,Mighty Aphrodite (1995),Comedy|Drama|Romance,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1,50,5.0,Georgia (1995),Drama,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,166534,4.0,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100832,610,168248,5.0,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100833,610,168250,5.0,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100834,610,168252,5.0,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df.pop('genres')

,genres
0,Adventure|Children|Fantasy
1,Comedy|Drama|Romance
2,Comedy|Romance
3,Comedy|Drama|Romance
4,Drama
...,...
100831,NaN
100832,NaN
100833,NaN
100834,NaN


In [9]:
df.head()

,userId,movieId,rating,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,Jumanji (1995),0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,Waiting to Exhale (1995),0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,Sabrina (1995),0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,1,47,5.0,Mighty Aphrodite (1995),0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,1,50,5.0,Georgia (1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df[["title", "year"]] = df['title'].str.extract(r'^(.*?)\s*\((\d{4})\)$')

In [11]:
df = df.dropna(subset=['year'])

In [12]:
df["year"] = df["year"].astype("int")

<ipython-input-12-8a658d5db1b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["year"] = df["year"].astype("int")


In [13]:
df_user_movie = df[["userId", "movieId"]]

In [14]:
df["year"].describe()

,year
count,77899.000000
mean,1989.779882
std,16.951942
min,1915.000000
25%,1986.000000
50%,1995.000000
75%,1999.000000
max,2017.000000


In [15]:
df_reordered = df[[col for col in df.columns if col != "Animation"] + ["Animation"]]
df_reordered.head()

,userId,movieId,rating,title,(no genres listed),Action,Adventure,Children,Comedy,Crime,...,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year,Animation
0,1,1,4.0,Jumanji,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1995,0
1,1,3,4.0,Waiting to Exhale,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1995,0
2,1,6,4.0,Sabrina,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1995,0
3,1,47,5.0,Mighty Aphrodite,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1995,0
4,1,50,5.0,Georgia,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1995,0


In [16]:
df_reordered.loc[:, "cos_vector"] = df_reordered.iloc[:, 5:23].apply(lambda row: row.to_list(), axis = 1)

<ipython-input-16-fbb9fa66ee5c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reordered.loc[:, "cos_vector"] = df_reordered.iloc[:, 5:23].apply(lambda row: row.to_list(), axis = 1)


In [17]:
df_reordered["cos_vector"][0]

[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [18]:
def int_to_list(input):
  return [int(input)]

df_reordered.loc[:, "Animation"] = df_reordered.loc[:, "Animation"].apply(int_to_list)

df_reordered.loc[:, "year"] = df_reordered.loc[:, "year"].apply(int_to_list)

df_reordered.head()

<ipython-input-18-10ffc23fd3f1>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[list([0]) list([0]) list([0]) ... list([0]) list([1]) list([1])]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_reordered.loc[:, "Animation"] = df_reordered.loc[:, "Animation"].apply(int_to_list)
<ipython-input-18-10ffc23fd3f1>:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[list([1995]) list([1995]) list([1995]) ... list([1971]) list([2014])
 list([1988])]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_reordered.loc[:, "year"] = df_reordered.loc[:, "year"].apply(int_to_list)


,userId,movieId,rating,title,(no genres listed),Action,Adventure,Children,Comedy,Crime,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year,Animation,cos_vector
0,1,1,4.0,Jumanji,0,0,1,1,0,0,...,0,0,0,0,0,0,0,[1995],[0],"[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,3,4.0,Waiting to Exhale,0,0,0,0,1,0,...,0,0,1,0,0,0,0,[1995],[0],"[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, ..."
2,1,6,4.0,Sabrina,0,0,0,0,1,0,...,0,0,1,0,0,0,0,[1995],[0],"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,1,47,5.0,Mighty Aphrodite,0,0,0,0,1,0,...,0,0,1,0,0,0,0,[1995],[0],"[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, ..."
4,1,50,5.0,Georgia,0,0,0,0,0,0,...,0,0,0,0,0,0,0,[1995],[0],"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [19]:
temp_df_1 = df_reordered.loc[:, "cos_vector"] + df_reordered.loc[:, "Animation"]
df_reordered.loc[:, "cos_vector"] = temp_df_1

temp_df = df_reordered.loc[:, "cos_vector"] + df_reordered.loc[:, "year"]

df_reordered.loc[:, "cos_vector"] = temp_df

df_reordered["cos_vector"][0]

[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1995]

In [20]:
df.loc[:, "cos_vector"] = df_reordered.loc[:, "cos_vector"]

df["cos_vector"][0]

<ipython-input-20-a2346bf9152f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "cos_vector"] = df_reordered.loc[:, "cos_vector"]


Index(['userId', 'movieId', 'rating', 'title', '(no genres listed)', 'Action',
       'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', 'year',
       'cos_vector'],
      dtype='object')

In [31]:
df.loc[:, "cos_vector"] = df.loc[:, "cos_vector"].apply(np.array)

df["cos_vector"][0]

array([[0.00000000e+00, 5.01252944e-04, 5.01252944e-04, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.01252944e-04,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.99999623e-01]])

In [22]:
df.loc[:, "cos_vector"] = df.loc[:, "cos_vector"].apply(lambda x: np.reshape(x, (1, -1)))

In [23]:
df.loc[:, "cos_vector"] = df.loc[:, "cos_vector"].apply(lambda x: normalize(x, norm="l2"))

In [32]:
df.loc[:, "cos_vector"] = df.loc[:, "cos_vector"].apply(lambda x: np.reshape(x, (-1)))

In [34]:
df.loc[:, "cos_vector"] = df.loc[:, "cos_vector"].apply(lambda x: x.tolist())

In [35]:
df["cos_vector"][0]

[0.0,
 0.0005012529439188763,
 0.0005012529439188763,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0005012529439188763,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9999996231181583]

In [ ]:
svd = SVD()

reader = Reader(rating_scale = (1, 5))

data = Dataset.load_from_df(df[["userId", "movieId", "rating"]], reader=reader)

trainset, testset = train_test_split(data, test_size = 0.2)

svd.fit(trainset)
pred = svd.test(testset)

accuracy.rmse(pred)

RMSE: 0.8833


0.8832749618771059

In [ ]:
with open("svd_model.pkl", "wb") as f:
  pickle.dump(svd, f)